In [146]:
import pandas as pd 
import numpy as np
import re
# on va extraire le csv 
df = pd.read_csv("czech.csv", header = 0, sep = "|")
print(df)

                SITE_NAME   INVOICE_NAME       BEGIN         END KWH_PERIOD  \
0      12345_GEO (0001BB)     1300082549  01/01/2018  31/01/2018      2 447   
1      12345_GEO (0001BB)     1300082549  01/02/2018  28/02/2018      2 211   
2      12345_GEO (0001BB)     1300082549  01/03/2018  31/03/2018      2 450   
3      12345_GEO (0001BB)     1300082549  01/04/2018  30/04/2018      2 431   
4      12345_GEO (0001BB)     1300082549  01/05/2018  31/05/2018      2 574   
...                   ...            ...         ...         ...        ...   
51427  19767_GEO (0632KO)  FV1/055100030  30/08/2019  31/12/2019      3 028   
51428  19767_GEO (0632KO)      055100058  01/01/2020  31/03/2020      2 163   
51429  19767_GEO (0632KO)      055100113  01/04/2020  30/06/2020      2 160   
51430  19767_GEO (0632KO)      055100168  01/07/2020  30/09/2020      2 230   
51431  19883_GEO (1203KO)     7220278453  27/09/2019  31/12/2019      4 745   

      PERIOD_EUR  
0         316,46  
1         286

## 1. Formater le nom des antennes 

In [147]:
# Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".
# on va donc enlever tout ce qui n'est pas entre parenthèses avec un regex 
df["SITE_NAME"] = df["SITE_NAME"].str.extract(r"\((.*)\)").dropna()

## 2. Supprimer la colonne "INVOICE_NAME"

In [148]:
df = df.drop("INVOICE_NAME", axis = 1)
display(df)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...
51427,0632KO,30/08/2019,31/12/2019,3 028,"397,18"
51428,0632KO,01/01/2020,31/03/2020,2 163,"283,57"
51429,0632KO,01/04/2020,30/06/2020,2 160,"312,55"
51430,0632KO,01/07/2020,30/09/2020,2 230,"294,07"


## 3. Calculer le nombre de jours facturés par période

In [149]:
# Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année)
# Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.
# on va créer une colonne "DAYS_FACTURED"
# il faut donc pour chaque ligne faire l'opération de conversion de différence entre date début et date fin en nombre de jours
# d'abord on convertit les colonnes en format date
df["END"] = pd.to_datetime(df["END"], format='%d/%m/%Y')
df["BEGIN"] = pd.to_datetime(df["BEGIN"], format='%d/%m/%Y')
df["days_billed"] = (df["END"] - df["BEGIN"]).dt.days
display(df)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,days_billed
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",30
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82",27
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32",30
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00",29
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37",30
...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3 028,"397,18",123
51428,0632KO,2020-01-01,2020-03-31,2 163,"283,57",90
51429,0632KO,2020-04-01,2020-06-30,2 160,"312,55",90
51430,0632KO,2020-07-01,2020-09-30,2 230,"294,07",91


## 4. Consommation journalière de la période

In [150]:
# A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.
# pour cela il faut enlever les espaces de la consommation de période pour pouvoir transformer cette colonne de strings en colonne de float 
df["KWH_PERIOD"] = df["KWH_PERIOD"].str.replace(r"[ ]+", "")
df["KWH_PERIOD"] = df["KWH_PERIOD"].astype(float)
df.loc[df["days_billed"] != 0, "KWH_DAY"] = df["KWH_PERIOD"] / df["days_billed"]
display(df)

C:\Users\pauln\AppData\Local\Temp\ipykernel_1716\923071991.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["KWH_PERIOD"] = df["KWH_PERIOD"].str.replace(r"[ ]+", "")


,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,days_billed,KWH_DAY
0,0001BB,2018-01-01,2018-01-31,2447.0,"316,46",30,81.566667
1,0001BB,2018-02-01,2018-02-28,2211.0,"286,82",27,81.888889
2,0001BB,2018-03-01,2018-03-31,2450.0,"317,32",30,81.666667
3,0001BB,2018-04-01,2018-04-30,2431.0,"315,00",29,83.827586
4,0001BB,2018-05-01,2018-05-31,2574.0,"333,37",30,85.800000
...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3028.0,"397,18",123,24.617886
51428,0632KO,2020-01-01,2020-03-31,2163.0,"283,57",90,24.033333
51429,0632KO,2020-04-01,2020-06-30,2160.0,"312,55",90,24.000000
51430,0632KO,2020-07-01,2020-09-30,2230.0,"294,07",91,24.505495


## 5. Nombre de factures par antenne

In [151]:
# ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total
# on compte d'abord les occurrences 
bills_number = df["SITE_NAME"].value_counts()
# on fait correspondre ces occurrences aux bonnes valeurs dans une nouvelle colonne 
# La méthode map() permet de remplacer chaque valeur de la colonne "SITE_NAME" par la valeur correspondante dans le dictionnaire bills_number. 
# Ainsi, chaque ligne de la colonne "bills_number" sera égale au nombre d'occurrences de la valeur correspondante dans la colonne "SITE_NAME".
df["bills_number"] = df["SITE_NAME"].map(bills_number) 
display(df)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,days_billed,KWH_DAY,bills_number
0,0001BB,2018-01-01,2018-01-31,2447.0,"316,46",30,81.566667,35
1,0001BB,2018-02-01,2018-02-28,2211.0,"286,82",27,81.888889,35
2,0001BB,2018-03-01,2018-03-31,2450.0,"317,32",30,81.666667,35
3,0001BB,2018-04-01,2018-04-30,2431.0,"315,00",29,83.827586,35
4,0001BB,2018-05-01,2018-05-31,2574.0,"333,37",30,85.800000,35
...,...,...,...,...,...,...,...,...
51427,0632KO,2019-08-30,2019-12-31,3028.0,"397,18",123,24.617886,4
51428,0632KO,2020-01-01,2020-03-31,2163.0,"283,57",90,24.033333,4
51429,0632KO,2020-04-01,2020-06-30,2160.0,"312,55",90,24.000000,4
51430,0632KO,2020-07-01,2020-09-30,2230.0,"294,07",91,24.505495,4


## 6. Coefficient de variation de la consommation par antenne 

In [152]:
# avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation
# Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie
# on crée une fonction qui signifie ici que pour une une valeur X on calcule la déviation standard avec degré de liberté 1 divisé par moyenne de X * 100
cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 

variation = []
for antenna in df["SITE_NAME"] : 
    # extraire les données pour l'antenne en cours
    kwh_day_antenna = df.loc[df["SITE_NAME"] == antenna, "KWH_DAY"] 
    # calculer le CV pour l'antenne en cours
    variation_antenna = cv(kwh_day_antenna) 
    # ajouter le résultat à la liste de variation
    variation.append(variation_antenna) 
print(variation_antenna)



KeyboardInterrupt: 

## 7. Créer un fichier CSV propre

In [ ]:
# on va mettre ça dans un nouveau csv 
clean_csv = df.to_csv("./clean_csv.csv", sep = ";", header = True)
